<a href="https://colab.research.google.com/github/HectorArielBaez/labo3-25/blob/main/generador_de_respuesta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

# Ruta a la carpeta con los CSV
ruta_carpeta = "/content/Salidas"  # ← CAMBIAR por tu ruta real
archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith(".csv")]

df_final = None

for archivo in archivos:
    path = os.path.join(ruta_carpeta, archivo)
    df = pd.read_csv(path)

    base = os.path.splitext(archivo)[0]

    # Renombrar columnas pred_tn y tn_feb con el nombre del archivo
    df = df.rename(columns={
        "pred_tn": f"{base}_pred_tn",
        "tn_feb": f"{base}_tn_feb",
        "real_tn": f"{base}_real_tn"  # evitar colisión si hay diferencias
    })

    if df_final is None:
        df_final = df
    else:
        df_final = pd.merge(df_final, df, on="product_id", how="inner")
# Eliminar duplicados si quedara alguno
df_final = df_final.drop_duplicates(subset=["product_id"])
# Guardar resultado
df_final.to_csv("combinado_por_product_id.csv", index=False)
print("Archivo guardado como 'combinado_por_product_id.csv'")


Archivo guardado como 'combinado_por_product_id.csv'


In [ ]:
import pandas as pd
import numpy as np

# Suponemos que df_final ya existe (resultado del merge anterior)
columna_id = df_final.columns[0]   # columna 0
columna_ref = df_final.columns[1]  # columna 1 (valor real)

resultados = []

for idx, fila in df_final.iterrows():
    ref = fila[columna_ref]
    errores = {}

    # Recorremos columnas 2, 4, 6, ..., es decir, columnas impares por índice
    for i in range(2, df_final.shape[1], 2):  # columnas 3, 5, 7,... (index 2, 4, 6...)
        pred = fila[i]
        if pd.notnull(pred):
            errores[i] = abs(ref - pred)

    if errores:
        col_mejor = min(errores, key=errores.get)  # índice de la mejor predicción
        col_siguiente = col_mejor + 1

        if col_siguiente < df_final.shape[1]:
            valor_siguiente = fila[col_siguiente]
        else:
            valor_siguiente = np.nan

        resultados.append({
            columna_id: fila[columna_id],
            "tn": valor_siguiente
        })

# Crear DataFrame final
df_resultado = pd.DataFrame(resultados)

# Guardar resultado si querés
df_resultado.to_csv("mejor_prediccion.csv", index=False)
print("Archivo 'mejor_prediccion.csv' generado correctamente.")


Archivo 'mejor_prediccion.csv' generado correctamente.


/tmp/ipython-input-16-2998919610.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = fila[i]
/tmp/ipython-input-16-2998919610.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  valor_siguiente = fila[col_siguiente]
